In [ ]:
#@title Pupil Mask Extraction and Reporting

import os
import cv2
import numpy as np
from PIL import Image
import glob

def extract_elliptical_pupil(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    best_mask = None
    max_area = 0

    for thresh_value in range(10, 100, 10):
        _, binary = cv2.threshold(blurred, thresh_value, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area > max_area:
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0

                if circularity > 0.5 and area > img.shape[0] * img.shape[1] * 0.01:
                    ellipse = cv2.fitEllipse(contour)
                    (center, axes, orientation) = ellipse
                    major_axis, minor_axis = axes

                    if 0.90 <= minor_axis / major_axis <= 1.11:
                        mask = np.zeros(img.shape, np.uint8)
                        cv2.ellipse(mask, ellipse, 1, -1)

                        white_ratio = np.sum(mask) / (mask.shape[0] * mask.shape[1])
                        if white_ratio < 0.5:
                            max_area = area
                            best_mask = mask

    return best_mask

def process_images(base_dir):
    input_dir = os.path.join(base_dir, 'images')
    output_dir = os.path.join(base_dir, 'masks')
    os.makedirs(output_dir, exist_ok=True)

    failed_images = []

    for img_path in glob.glob(os.path.join(input_dir, '*.jpg')):
        mask = extract_elliptical_pupil(img_path)

        if mask is None:
            print(f"Warning: Could not extract pupil from {img_path}")
            failed_images.append(img_path)
            continue

        base_name = os.path.splitext(os.path.basename(img_path))[0]
        gif_path = os.path.join(output_dir, f"{base_name}.gif")
        img_pil = Image.fromarray(mask * 255).convert('1')
        img_pil.save(gif_path, format='GIF')
        print(f"Saved mask for {base_name}")

    if failed_images:
        print("The following images could not have masks produced:")
        for img in failed_images:
            print(img)

base_directory = "data/val"
process_images(base_directory)


In [ ]:
#@title Apply Masks to Images

import os
import cv2
import numpy as np
from PIL import Image

def apply_masks(directory):
    images_dir = os.path.join(directory, 'images')
    masks_dir = os.path.join(directory, 'masks')
    applied_dir = os.path.join(directory, 'applied')

    os.makedirs(applied_dir, exist_ok=True)

    for filename in os.listdir(images_dir):
        if filename.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(images_dir, filename)
            mask_path = os.path.join(masks_dir, os.path.splitext(filename)[0] + '.gif')

            if not os.path.exists(mask_path):
                print(f'Mask for {filename} not found. Skipping.')
                continue

            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            pupil_mask = np.array(Image.open(mask_path).convert('L'))

            red_overlay = np.zeros_like(image)
            red_overlay[:, :, 2] = 255

            result = image.copy()
            red_area = pupil_mask == 255
            result[red_area] = cv2.addWeighted(image[red_area], 0.5, red_overlay[red_area], 0.5, 0)

            applied_output_path = os.path.join(applied_dir, filename)
            cv2.imwrite(applied_output_path, result)

directory = 'data/val'
apply_masks(directory)


In [ ]:
#@title Generate GIF from Processed Images

import os
from PIL import Image

def generate_gif(directory):
    applied_dir = os.path.join(directory, 'applied')
    applied_images = []

    for filename in sorted(os.listdir(applied_dir)):
        if filename.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(applied_dir, filename)
            applied_images.append(Image.open(image_path))

    output_gif_path = os.path.join(directory, 'output.gif')
    if applied_images:
        applied_images[0].save(output_gif_path, save_all=True, append_images=applied_images[1:], duration=1000//15, loop=0)

# Example usage
directory = 'data/val'
generate_gif(directory)
